<center><h1>Shoppee Pandas data manipulation questions</h1></center>

## Question 1:

#### a. Use pandas function to read the Test_Pandas.xlsx file in.

In [64]:
import pandas as pd
data = pd.read_excel('data.xlsx')

#### b. Assign all column names to a variable named “columns”

In [65]:
columns = data.columns

---- 
## 2.	Use pandas function to find:

#### a.	How many unique shops are in the dataset?

In [66]:
data.shopid.nunique()

7856

#### b.	How many unique preferred and cross border shops are in the dataset?

In [67]:
data[(data.cb_option == 1) & (data.is_preferred == 1)].shopid.nunique()

158

#### c.	How many products have zero sold count?

In [68]:
data[data.sold_count == 0].itemid.count()

438092

#### d.	How many products were created in the year 2018?

In [69]:
data[data.item_creation_date.dt.year == 2018].itemid.count()

177407

In [91]:
len(data[data.item_creation_date.dt.year == 2018])

177407

---
## 3.	Use pandas function to find:

#### a.	Top 3 Preferred shops’ shopid that have the largest number of unique products

In [70]:
data[data.is_preferred == 1].groupby('shopid').itemid.count().sort_values(ascending=False).head(3)

shopid
43384791    2034
6072241     1998
26974701    1037
Name: itemid, dtype: int64

#### b.	Top 3 Categories that have the largest number of unique cross-border products

In [71]:
data[data.cb_option == 1].groupby('category').itemid.count().sort_values(ascending=False).head(3)

category
Women's Apparel     80494
Mobile & Gadgets    58760
Men's Wear          51795
Name: itemid, dtype: int64

---
## 4. Find Top 3 shopid with the highest revenue (Assumption: the product price has not been changed.) 

In [72]:
# creating a new column revenue which is atttained by taking the price of the product x each count of it sold.
data['revenue'] = data.price * data.sold_count

In [73]:
# Grouping the shop id and then using the sum aggregate to attain total revenue per shop.
data.groupby('shopid').revenue.sum().sort_values(ascending=False).head(3)

shopid
18796234    177897.79
30769353     63491.39
30628794     39852.00
Name: revenue, dtype: float64

---
## 5.	Find the number of products that have more than 3 variations (do not include products with 3 or fewer variations)

In [94]:
df[data.item_variation.apply(lambda x: x.count(',')) > 3].itemid.count()

213585

In [74]:
import numpy as np
data['3var'] = np.where(data.item_variation.apply(lambda x: x.count(',')) > 3, 'yes', 'no')

#### Breaking down the one-liner code above

In [75]:
# Since each variation is separated by a comma count the commas using the apply function.
# As you can see the first 4 rows have 0 variations and row 4 has 5 variation
data.item_variation.apply(lambda x: x.count(',')).head(5)

0    0
1    0
2    0
3    0
4    5
Name: item_variation, dtype: int64

In [76]:
# applying the filter of bigger than 3 I get boolean values in return
# row 4 has 5 variations and that is more than 3 so it returns True
(data.item_variation.apply(lambda x: x.count(',')) > 3).head(5)

0    False
1    False
2    False
3    False
4     True
Name: item_variation, dtype: bool

In [77]:
# np.where works like this : np.where(boolean values, x, y)
# For the boolean values it returns x (the second parameter) if the boolean value is True
# and it returns y if the boolean values is False.
# You can see that the first 4 values are 'no' and the 5th value is 'yes' 
np.where(data.item_variation.apply(lambda x: x.count(',')) > 3, 'yes', 'no')[:5]

array(['no', 'no', 'no', 'no', 'yes'], dtype='<U3')

In [78]:
# Translate these values into a new column in the data frame 
data['3var'] = np.where(data.item_variation.apply(lambda x: x.count(',')) > 3, 'yes', 'no')

In [79]:
# You can observe the new column 3var(iations) on the right.
data.head(5)

,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date,revenue,3var
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42,0.0,no
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37,0.0,no
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13,0.0,no
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07,0.0,no
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.0,300,Women's Apparel,0,0,0,2016-05-09 01:14:34,0.0,yes


--- 
## 6.	Use pandas function to identify duplicated listings within each shop (If listing A and B in shop S have the exactly same product title, product detailed description, and price, both listing A and B are considered as duplicated listings)

#### a.	Mark those duplicated listings with True otherwise False and store the marking result in a new column named “is_duplicated”

In [80]:
data['duplicated'] = data.duplicated(subset=['item_name','item_description','price'], keep=False)

Breaking down the one line of code above

In [81]:
# .duplicated() gives you back boolean values checking for duplicates within the dataframe. 
data.duplicated().head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [82]:
# Since the instructions specifies that it's only a duplicate where 3 columns are concerned,
# (1) product title, (2) product description, (3) Price I have to constrain the .duplicated method to these
# 3 columns only. 
# you can see we have a True at row 10
data.duplicated(subset=['item_name','item_description','price']).head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9     True
dtype: bool

In [83]:
# The way .duplicated method works is to only specify True at the second occurence of the duplicate.
# The first occurence is not marked as True
# If you want all duplicates to be marked as True you have to specify the argument keep=False
# As compared to the prior example you can see that row 8 returns True thus it's the first occurnece 
    # while row 9 is the second occurence. 
data.duplicated(subset=['item_name','item_description','price'], keep=False).head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8     True
9     True
dtype: bool

In [84]:
# Since the instruction wants the answer in Boolean values I can attached the answer 
    # directly into a new pandas column
data['duplicated'] = data.duplicated(subset=['item_name','item_description','price'], keep=False)

In [85]:
# you can observe the duplicated column here
data.head()

,itemid,shopid,item_name,item_description,item_variation,price,stock,category,cb_option,is_preferred,sold_count,item_creation_date,revenue,3var,duplicated
0,1925574,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355333 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:24:42,0.0,no,False
1,1925617,210000,Golden mobile numbers,Unregistered prepaid cards1) 93355888 selling...,{},400.0,1,Mobile & Gadgets,0,0,0,2015-10-01 00:26:37,0.0,no,False
2,943600,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-25 21:02:13,0.0,no,False
3,1064405,210000,Golden Mobile Numbers,Unregistered prepaid cards. Can port to post p...,{},8.0,1,Mobile & Gadgets,0,0,0,2015-08-30 20:16:07,0.0,no,False
4,20046620,760000,101% AUTHENTIC BASEBALL CAPS,"PREORDER Takes about 23 weeks to arrive, will ...","{NEWERA BLACK: 35.0, NIKE SWOOSH DENIM: 35.0, ...",35.0,300,Women's Apparel,0,0,0,2016-05-09 01:14:34,0.0,yes,False


#### b.	Find duplicated listings that has less than 2 sold count and store the result in a new excel file named “duplicated_listings.xlsx”

In [86]:
# getting only the duplicated rows
duplicate = data[data['duplicated']]

# storing only those duplicates which have below 2 sold counts
duplicated_listings = duplicate[duplicate.sold_count < 2]

Storing the result in an excel format. Don't want this to actually execute it takes up too much time.

`duplicated_listings.to_excel('duplicated_listings.xlsx')`

#### c.	Find the preferred shop shopid that have the most number of duplicated listin

In [87]:
# accessing the duplicated list and filtering only the preferred shops
# Since shops ids are in focus group it
# Do an aggregate count to get the sum of the duplicates per shop
duplicate[duplicate.is_preferred == 1].groupby('shopid').duplicated.count().sort_values(ascending=False).head()

shopid
43384791    2009
6072241     1250
26974701     635
8090602      615
37983742     436
Name: duplicated, dtype: int64